In [8]:
# Import Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm
from lightgbm import LGBMClassifier

In [9]:
lightgbm.__version__

'3.3.2'

In [10]:
df = pd.read_csv("data/train.csv")
df["ID"] = df["filename"].str.split(".",expand=True)[0]
df=df.drop("filename",axis=1)
ss = pd.read_csv("data/SampleSubmission.csv")
tdf = pd.read_csv("data/test.csv")

In [11]:
num_cols=["access_success_rate","resource_utilition_rate","TA","bler","cqi","mcs"]

In [12]:
df["NE ID ID COUNT"] = df.groupby("NE ID")["ID"].transform("nunique")
tdf["NE ID ID COUNT"] = tdf.groupby("NE ID")["ID"].transform("nunique")

In [13]:
cat_cols=["A","B","C","ID","NE ID"]

In [15]:
# Extract Time Features
for x in [df,tdf]:
    x[["A","C"]] = x.ID.str.split("_",expand=True)
    x[["A","B"]] = x.A.str.split("-",expand=True)
    x["A"] = x.A.str.split("B",expand=True)[1]
    for col in ["A","B","C"]:
        x[col] = x[col].astype("int")
    x.endTime=pd.to_datetime(x.endTime)
    x["month"] = x.endTime.dt.month
    x["day"] = x.endTime.dt.day
    x["dow"] = x.endTime.dt.day_of_week.astype("int")
    x["hour"] = x.endTime.dt.hour
    x["ts"] = x.endTime.values.astype("int")
    x["ts"] = x.ts/1000000000

In [16]:
df=df.reset_index()

In [17]:
tdf=tdf.reset_index()

In [18]:
df["test"]=0
tdf["test"]=1

In [19]:
df=df.append(tdf[df.columns])

<ipython-input-19-488dac3fc5af>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(tdf[df.columns])


In [20]:
df = df.sort_values(["ID","endTime"])

In [21]:
df["IC"] = df.groupby(["ID"])["ID"].transform("count")
df["NEC"] = df.groupby(["NE ID"])["NE ID"].transform("count")
df["AC"] = df.groupby(["A"])["A"].transform("count")
df["BC"] = df.groupby(["B"])["B"].transform("count")
df["CC"] = df.groupby(["C"])["C"].transform("count")

In [22]:
df["ICr"] = df.groupby(["ID"])["index"].transform("min")
df["ICr"] = df["index"]-df["ICr"]  

In [23]:
df["fr"]=df.fault_duration*df.relation
df["nfr"]=df.groupby(["ID"])["fr"].shift(-1)

In [24]:
df["next_data_rate"] = df.groupby(["ID"])["data_rate"].shift(-1)
df["next_fault_duration"] = df.groupby(["ID"])["fault_duration"].shift(-1)
df["next_relation"] = df.groupby(["ID"])["relation"].shift(-1)

In [25]:
# Extract Lag Features
for i in range(1,43,1):
    for col in ["data_rate"]:
        df[f"last_{col}_{i}"] = df.groupby(["ID"])[col].shift(i)
        df[f"last_{col}_vs_{i}"] = (df[col]+0.1)/(df[f"last_{col}_{i}"]+0.1)
        if i > 1:
            df[f"last_{col}_{i}_vs_{i-1}"] = (df[f"last_{col}_{i}"]+0.1)/(df[f"last_{col}_{i-1}"]+0.1)

<ipython-input-25-68d9d00731d2>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"last_{col}_{i}"] = df.groupby(["ID"])[col].shift(i)
<ipython-input-25-68d9d00731d2>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"last_{col}_vs_{i}"] = (df[col]+0.1)/(df[f"last_{col}_{i}"]+0.1)
<ipython-input-25-68d9d00731d2>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

<ipython-input-25-68d9d00731d2>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"last_{col}_{i}"] = df.groupby(["ID"])[col].shift(i)
<ipython-input-25-68d9d00731d2>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"last_{col}_vs_{i}"] = (df[col]+0.1)/(df[f"last_{col}_{i}"]+0.1)
<ipython-input-25-68d9d00731d2>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

In [26]:
tdf = df[df["test"]==1]
tdf = tdf[tdf["next_fault_duration"]>0]
df = df[df["test"]==0]


In [27]:
df["class"] = (df["data_rate"]>df["next_data_rate"]).astype("int")
df["class"].describe()

count    908922.000000
mean          0.483683
std           0.499734
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: class, dtype: float64

In [28]:
target = "next_data_rate"

In [29]:
drop_cols = [
    "NE ID","endTime","ID",target, "class","test"
]
cols = df.drop(drop_cols,axis=1).columns


In [30]:
df["count_available_features"] = df[cols].count(axis=1)
tdf["count_available_features"] = tdf[cols].count(axis=1)

In [31]:
select_cols = tdf[tdf.count_available_features==31]
select_cols = list(select_cols[cols].dropna(axis=1).columns)
len(select_cols)

31

- # Model 1 - Train With All Features

In [32]:
model1 = LGBMClassifier(n_estimators=1000)
tdf["class"]=model1.fit(df[cols],df["class"]).predict_proba(tdf[cols])[:,1]
tdf["class"]= (tdf["class"]>0.4).astype("int")
x=tdf.copy()
label=ss.columns[1]
x[label]=x["class"]
pred1=x[ss.columns]
pred1[label]=pred1[label].astype("int")
pred1

<ipython-input-32-0ae77f146df6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred1[label]=pred1[label].astype("int")


,ID,data_rate_t+1_trend
2,B0017-25_24,1
5,B0017-25_25,0
7,B0017-25_26,0
11,B0017-25_27,1
144,B0017-32_1,1
...,...,...
174665,B0122-43_1,1
174713,B0122-43_2,0
174776,B0122-45_1,1
174824,B0122-45_2,1


In [33]:
cols=select_cols
tdf=tdf[tdf.count_available_features==len(cols)]
len(tdf), len(cols)

(131, 31)

- # Model 2 - Train Without Lag Features

In [34]:

model2 = LGBMClassifier(n_estimators=1000)
tdf["class"]=model2.fit(df[cols],df["class"]).predict_proba(tdf[cols])[:,1]
tdf["class"]= (tdf["class"]>0.4).astype("int")
x=tdf.copy()
label=ss.columns[1]
x[label]=x["class"]
pred2=x[ss.columns]
pred2[label]=pred2[label].astype("int")
pred2

<ipython-input-34-f81dbfb24bc0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred2[label]=pred2[label].astype("int")


,ID,data_rate_t+1_trend
5,B0017-25_25,0
155,B0017-32_12,1
173,B0017-32_14,1
176,B0017-32_15,0
187,B0017-32_17,0
...,...,...
127716,B0093-23_30,1
128085,B0093-25_27,0
128130,B0093-25_30,1
128246,B0093-25_39,0


In [36]:
# Append Predictions
pred=pred1.append(pred2)
pred=pred.drop_duplicates("ID",keep="last")
pred.to_csv('prediction.csv',index=False)
pred.describe()

<ipython-input-36-c59d10c5da20>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred=pred1.append(pred2)


,data_rate_t+1_trend
count,1932.000000
mean,0.699793
std,0.458467
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000
